In [27]:
# !wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch/minsearch.py

In [28]:
# !wget https://rawgithubusercontent.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json

In [29]:
import minsearch

In [30]:
import json

In [31]:
with open('../data/documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [32]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [33]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [34]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [35]:
index.fit(documents)

In [36]:
from openai import OpenAI

In [37]:
client = OpenAI()

In [38]:
def search(query):
    boost = {'question': 3.0, 'section':0.5}
    results = index.search(
        query = query,
        # filter_dict = {'course': 'data-engineering-zoomcamp'},
        boost_dict = boost,
        num_results=5
    )
    return results

In [39]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTIONS based
    on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    If the CONTEXT doesn't contain the answer, output NONE
    
    QUESTION: {question}
    
    CONTEXT:
    {context}
    """.strip()
    context = ""
    
    for doc in search_results:
        context += f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    
    return prompt

In [40]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4.1-nano',
        messages = [{"role": "user", "content":prompt}]
    )
    return response.choices[0].message.content

In [63]:
query = 'what else  do i need to do other than finishing the project to get a certificate?'
search_results = search(query)
prompt = build_prompt(query, search_results)
answer = llm(prompt)

In [64]:
answer

'You need to review 3 peers’ Projects by the deadline to get a certificate.'

In [56]:
search_results

[{'text': 'In the project directory, run:\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java',
  'section': 'Module 6: streaming with kafka',
  'question': 'Java Kafka: How to run producer/consumer/kstreams/etc in terminal',
  'course': 'data-engineering-zoomcamp'},
 {'text': "Solution from Alexey: create a virtual environment and run requirements.txt and the python files in that environment.\nTo create a virtual env and install packages (run only once)\npython -m venv env\nsource env/bin/activate\npip install -r ../requirements.txt\nTo activate it (you'll need to run it every time you need the virtual env):\nsource env/bin/activate\nTo deactivate it:\ndeactivate\nThis works on MacOS, Linux and Windows - but for Windows the path is slightly different (it's env/Scripts/activate)\nAlso the virtual environment should be created only to run the python file. Docker images should first all be up and running.",
  'section': 'Module 6: streaming wi